In [1]:
#@title Install required packages
!uv pip install -U pse # proxy structuring engine
!uv pip install sentencepiece
!uv pip install accelerate
!uv pip install transformers
!uv pip install torch
!uv pip install numpy
!uv pip install bitsandbytes
!uv pip install sentencepiece
!uv pip install protobuf
!uv pip install -U tqdm
!uv pip install ipywidgets

Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Resolved 26 packages in 171ms                                        
Audited 26 packages in 0.13ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 2ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 2ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1ms
Using Python 3.12.7 environment at /Users/jckwind/Documents/proxy-structuring-engine/.venv
Audited 1 package in 1

In [6]:
# @title Setup Llama 3.2 1B
import torch
from transformers import AutoTokenizer, LlamaForCausalLM

from pse.engine.structuring_engine import StructuringEngine
from pse.util.torch_mixin import PSETorchMixin


class PSE_Torch(PSETorchMixin, LlamaForCausalLM):
    pass


model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PSE_Torch.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.config.pad_token_id = model.config.eos_token_id[0]
if model.generation_config:
    model.generation_config.pad_token_id = model.config.eos_token_id[0]

In [7]:

#@title Create engine and test json generation
import json

SIMPLE_JSON_SCHEMA = {
    "type": "object",
    "properties": {"value": {"type": "number"}},
    "required": ["value"],
}
model.engine = StructuringEngine(tokenizer)
model.engine.configure(SIMPLE_JSON_SCHEMA)
prompt = "Please generate a json object with the value 9.11, with the following schema:\n"
prompt += json.dumps(SIMPLE_JSON_SCHEMA, indent=2)
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
output = model.generate(
    input_ids,
    do_sample=True,
    temperature=1.0,
)
print("Output:\n" + 100 * "-")
print(tokenizer.decode(output[0]))

Output:
----------------------------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Please generate a json object with the value 9.11, with the following schema:
{
  "type": "object",
  "properties": {
    "value": {
      "type": "number"
    }
  },
  "required": [
    "value"
  ]
}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{
  "value": 9.11
}
